# MLTagger

## Background

This repo is based on the paper [Zero-shot Sequence Labeling: Transferring Knowledge from Sentences to Tokens](https://arxiv.org/pdf/1805.02214.pdf) by Marek Rei and Anders Søgaard. The original code repositary can be found [here](https://github.com/marekrei/mltagger).

## Getting Started

To begin, train the model with experiment.py where it takes in a configuration file that contains the necessary hyperparameters and file paths. The script is dependent on two other files ([model.py](https://github.com/DerekChia/mltagger/blob/master/src/model.py) and [evaluator.py](https://github.com/DerekChia/mltagger/blob/master/src/evaluator.py)) where the MLTModel and MLTEvaluator objects are initialised and updated during runtime. Since we are using Jupyter Notebook, these two files are imported using model.ipynb and evaluator.ipynb.

Reference: [src/experiment.py](https://github.com/DerekChia/mltagger/blob/master/src/experiment.py)

In [1]:
%run model.ipynb
%run evaluator.ipynb

In [2]:
import sys
import collections
import numpy
import random
import math
import os
import gc
import tensorflow as tf

try:
    import ConfigParser as configparser
except:
    import configparser

# Replaced by %run model.ipynb and evaluator.ipynb
# from model import MLTModel
# from evaluator import MLTEvaluator

- read_input_files(file_paths, max_sentence_length=-1) return *sentences*

- parse_config(config_section, config_path) return *config*

- is_float(value) return *{True, False}*

### read_input_files
This function takes in the input file(s) path and return sentences in the following format. Note that the input files are expected to be in TSV format, CoNLL style.

>sentence = [['Not', 'c'], ['only', 'c'], ['as', 'c'], ['a', 'c'], ['hobby', 'c'], ['.', 'c']]

>sentences = [
> [['Not', 'c'], ['only', 'c'], ['as', 'c'], ['a', 'c'], ['hobby', 'c'], ['.', 'c']],
> [['They', 'c'], ['use', 'c'], ['computers', 'c'], ['for', 'c'], ['their', 'c'], ['works', 'i'], ['.', 'c']]
>]

In [ ]:
def read_input_files(file_paths, max_sentence_length=-1):
    """
    Reads input files in whitespace-separated format.
    Will split file_paths on comma, reading from multiple files.
    """
    sentences = []
    line_length = None
    # file_path might contain multiple files, split them and iterate through
    for file_path in file_paths.strip().split(","):
        with open(file_path, "r") as f:
            sentence = []
            for line in f:
                line = line.strip()
                # Ensure that line contains character, else this is an indicator for newline
                if len(line) > 0:
                    line_parts = line.split()
                    # Check if input file has both word and label
                    # Might not be a necessary check during inference since 
                    # input may not have a ground truth (second column)
                    # assert(len(line_parts) >= 2), line
                    # assert(len(line_parts) == line_length or line_length == None)
                    # line_length = len(line_parts)
                    sentence.append(line_parts)
                # If line has no character (i.e. empty) and length of previous sentence is more than zero,
                # push the previous sentence into the sentences list and anticipate new sentence.
                elif len(line) == 0 and len(sentence) > 0:
                    if max_sentence_length <= 0 or len(sentence) <= max_sentence_length:
                        sentences.append(sentence)
                    sentence = []
            # Not in use
            if len(sentence) > 0:
                if max_sentence_length <= 0 or len(sentence) <= max_sentence_length:
                    sentences.append(sentence)
    return sentences

### parse_config
This function reads configuration from the input `config_path` and returns a dictionary where it tries to guess the correct datatype for each config value. `config_section` is used as a starting key [config].

In [ ]:
def parse_config(config_section, config_path):
    """
    Reads configuration from the file and returns a dictionary.
    Tries to guess the correct datatype for each of the config values.
    """
    config_parser = configparser.SafeConfigParser(allow_no_value=True)
    config_parser.read(config_path)
    config = collections.OrderedDict()
    for key, value in config_parser.items(config_section):
        if value is None or len(value.strip()) == 0:
            config[key] = None
        elif value.lower() in ["true", "false"]:
            config[key] = config_parser.getboolean(config_section, key)
        elif value.isdigit():
            config[key] = config_parser.getint(config_section, key)
        elif is_float(value):
            config[key] = config_parser.getfloat(config_section, key)
        else:
            config[key] = config_parser.get(config_section, key)
    return config

### is_float
This function checks if the parameter is of type float.

In [ ]:
def is_float(value):
    """
    Check in value is of type float()
    """
    try:
        float(value)
        return True
    except ValueError:
        return False

### create_batches_of_sentence_ids
Based on the input parameters:

- If `batch_equal_size` is `True`, group sentences into batches by IDs based on their length, according to a limit of `max_batch_size`. For example, if there are 64 sentences with length 10 (i.e. 64 sentences each with 10 characters in total) and `max_batch_size` is set to `32`, these 64 sentences will be batched together into 2 batches.
    ```
    Example:
    Length of each sentence, [ID of each sentence]
    10, [   [0, ... , 31]
    10,     [32, ... , 63]
    25,     [2553, 2680]
    43,     [142, 305, 1490, 1775, 1973]
    27,     [194, 197, 245, 348, 359]
    XX,     [...]    ]
    ```
- If `batch_equal_size` is `False`, each batch will contain a fixed number of sentences (`max_batch_size`).
    ```
    Example:
    [ID of each sentence]
    [
        [0, 1, 2, 3, 4]
        [5, 6, 7, 8, 9]
        [10, 11, 12, 13, 14]
    ]
    ```

Return: 
- List of lists with sentences ids.

Default value:
- `batch_equal_size` = `True`
- `max_batch_size` = `32`

In [ ]:
def create_batches_of_sentence_ids(sentences, batch_equal_size, max_batch_size):
    """
    Groups together sentences into batches
    If max_batch_size is positive, this value determines the maximum number of sentences in each batch.
    If max_batch_size has a negative value, the function dynamically creates the batches such that each batch contains abs(max_batch_size) words.
    Returns a list of lists with sentences ids.
    """
    if batch_equal_size == True:
        sentence_ids_by_length = collections.OrderedDict()
        for _id, sentence in enumerate(sentences):
            length = len(sentence)
            if length not in sentence_ids_by_length:
                sentence_ids_by_length[length] = []
                sentence_ids_by_length[length].append(_id)
            else:
                sentence_ids_by_length[length].append(_id)
        # Fitting each batch to the size defined in max_batch_size
        for sentence_length in sentence_ids_by_length:
            for i in range(0, len(sentence_ids_by_length[sentence_length]), max_batch_size):
                batches_of_sentence_ids.append(sentence_ids_by_length[sentence_length][i : i + max_batch_size])
    else:
        batch = []
        for i in range(len(sentences)):
            if len(batch) != max_batch_size:
                batch.append(i)
            else:
                batches_of_sentence_ids.append(batch)
                batch = []
                batch.append(i)
    return batches_of_sentence_ids

"""Original Code Below"""
#     batches_of_sentence_ids = []
#     if batch_equal_size == True:
#         sentence_ids_by_length = collections.OrderedDict()
#         sentence_length_sum = 0.0
#         for i in range(len(sentences)):
#             length = len(sentences[i])
#             if length not in sentence_ids_by_length:
#                 sentence_ids_by_length[length] = []
#             sentence_ids_by_length[length].append(i)

#         for sentence_length in sentence_ids_by_length:
#             if max_batch_size > 0:
#                 batch_size = max_batch_size
#             else:
#                 batch_size = int((-1 * max_batch_size) / sentence_length)

#             for i in range(0, len(sentence_ids_by_length[sentence_length]), batch_size):
#                 batches_of_sentence_ids.append(sentence_ids_by_length[sentence_length][i:i + batch_size])
#     else:
#         current_batch = []
#         max_sentence_length = 0
#         for i in range(len(sentences)):
#             current_batch.append(i)
#             if len(sentences[i]) > max_sentence_length:
#                 max_sentence_length = len(sentences[i])
#             if (max_batch_size > 0 and len(current_batch) >= max_batch_size) \
#               or (max_batch_size <= 0 and len(current_batch)*max_sentence_length >= (-1 * max_batch_size)):
#                 batches_of_sentence_ids.append(current_batch)
#                 current_batch = []
#                 max_sentence_length = 0
#         if len(current_batch) > 0:
#             batches_of_sentence_ids.append(current_batch)
#     return batches_of_sentence_ids

### process_sentences

Function processes all sentences with evaluator object and return evaluation metrics. Function will trigger `process_batch` function by feeding in batch of sentences and learning rate configured.

Return:
- evaluator object with results populated

Default value:
- `garbage_collection` = `False`
- `batch_equal_size` = `True`
- `max_batch_size` = `32`

In [3]:
def process_sentences(epoch, data, model, is_training, learningrate, config, name):
    """
    Process all the sentences with the labeler, return evaluation metrics.
    """
    evaluator = MLTEvaluator(config)
    
    # From data, create batches of sentences ids based batch_equal_size flag.
    # batch_equal_size = False, max_batch_size = 32
    batches_of_sentence_ids = create_batches_of_sentence_ids(data, config["batch_equal_size"], config["max_batch_size"])
    
    # Randomly shuffle data AGAIN if this is training data. 
    # Question: Why shuffle again when data_train was shuffled in run_experiment() before this?
    # See random.shuffle(data_train)
    if is_training == True:
        random.shuffle(batches_of_sentence_ids)
    
    # sentence_ids_in_batch refers to each batch of sentence_ids
    # E.g. sentence_ids_in_batch = [10, 11, 12, 13, 14]
    for count, sentence_ids_in_batch in enumerate(batches_of_sentence_ids):        
        # Build batch from data. Each batch now contains sentences (not ID)
        # For every sentence ID (i) in sentence_ids, get the actual sentence data[i]
        # batch = [
        # [['Not', 'c'], ['only', 'c'], ['as', 'c'], ['a', 'c'], ['hobby', 'c'], ['.', 'c']], # First sentence
        # [['They', 'c'], ['use', 'c'], ['computers', 'c'], ['for', 'c'], ['their', 'c'], ['works', 'i'], ['.', 'c']] # Second sentence
        #]
        batch = [data[i] for i in sentence_ids_in_batch]
        
        # Get cost, predicted labels and probs for each batch
        print('############### Epoch', epoch + 1,'Batch', count + 1, 'of', len(batches_of_sentence_ids) , '###############')
        cost, sentence_scores, token_scores_list = model.process_batch(batch, is_training, learningrate)
        
        # Append cost, predicted labels and probs to the evaluator object
        evaluator.append_data(cost, batch, sentence_scores, token_scores_list)
    
        # Not in use. garbage_collection defaults to False.
        while config["garbage_collection"] == True and gc.collect() > 0:
            pass

        results = evaluator.get_results(name)
        for key in results:
            print(key + ": " + str(results[key]))

    return results

### run_experiment



In [4]:
def run_experiment(config_path):
    config = parse_config("config", config_path)
    temp_model_path = config_path + ".model"
    if "random_seed" in config:
        random.seed(config["random_seed"])
        numpy.random.seed(config["random_seed"])

    # To print everything in config - not needed for now
    # for key, val in config.items():
    #     print(str(key) + ": " + str(val))

    data_train, data_dev, data_test = None, None, None
    if config["path_train"] != None and len(config["path_train"]) > 0:
        data_train = read_input_files(config["path_train"], config["max_train_sent_length"])
    if config["path_dev"] != None and len(config["path_dev"]) > 0:
        data_dev = read_input_files(config["path_dev"])
    if config["path_test"] != None and len(config["path_test"]) > 0:
        data_test = []
        for path_test in config["path_test"].strip().split(":"):
            data_test += read_input_files(path_test)
    
    model = MLTModel(config)
    model.build_vocabs(data_train, data_dev, data_test, config["preload_vectors"])
    model.construct_network()
    model.initialize_session()
    if config["preload_vectors"] != None:
        model.preload_word_embeddings(config["preload_vectors"])

    print("parameter_count: " + str(model.get_parameter_count()))
    print("parameter_count_without_word_embeddings: " + str(model.get_parameter_count_without_word_embeddings()))

    if data_train != None:
        model_selector = config["model_selector"].split(":")[0]
        model_selector_type = config["model_selector"].split(":")[1]
        best_selector_value = 0.0
        best_epoch = -1
        learningrate = config["learningrate"]
        for epoch in range(config["epochs"]):
            print("EPOCH: " + str(epoch))
            print("current_learningrate: " + str(learningrate))
            random.shuffle(data_train)

            results_train = process_sentences(epoch, data_train, model, is_training=True, learningrate=learningrate, config=config, name="train")

            if data_dev != None:
                results_dev = process_sentences(epoch, data_dev, model, is_training=False, learningrate=0.0, config=config, name="dev")

                if math.isnan(results_dev["dev_cost_sum"]) or math.isinf(results_dev["dev_cost_sum"]):
                    raise ValueError("Cost is NaN or Inf. Exiting.")

                if (epoch == 0 or (model_selector_type == "high" and results_dev[model_selector] > best_selector_value) 
                               or (model_selector_type == "low" and results_dev[model_selector] < best_selector_value)):
                    best_epoch = epoch
                    best_selector_value = results_dev[model_selector]
                    model.saver.save(model.session, temp_model_path, latest_filename=os.path.basename(temp_model_path)+".checkpoint")
                print("best_epoch: " + str(best_epoch))

                if config["stop_if_no_improvement_for_epochs"] > 0 and (epoch - best_epoch) >= config["stop_if_no_improvement_for_epochs"]:
                    break

                if (epoch - best_epoch) > 3:
                    learningrate *= config["learningrate_decay"]

            while config["garbage_collection"] == True and gc.collect() > 0:
                pass

        if data_dev != None and best_epoch >= 0:
            # loading the best model so far
            model.saver.restore(model.session, temp_model_path)
            os.remove(temp_model_path+".checkpoint")
            os.remove(temp_model_path+".data-00000-of-00001")
            os.remove(temp_model_path+".index")
            os.remove(temp_model_path+".meta")

    if config["save"] is not None and len(config["save"]) > 0:
        model.save(config["save"])

    if config["path_test"] is not None:
        i = 0
        for path_test in config["path_test"].strip().split(":"):
            data_test = read_input_files(path_test)
            results_test = process_sentences(epoch, data_test, model, is_training=False, learningrate=0.0, config=config, name="test"+str(i))
            i += 1

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: The SafeConfigParser class has been renamed to ConfigParser in Python 3.2. This alias will be removed in future versions. Use ConfigParser directly instead.


n_words: 13470
n_chars: 97
n_singletons: 7078
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
n_preloaded_embeddings: 9469
parameter_count: 5934452
parameter_count_without_word_embeddings: 1893452
EPOCH: 0
current_learningrate: 1.0
############### Epoch 1 Batch 1 of 898 ###############
train_cost_avg: 0.29047760367393494
train_cost_sum: 9.295283317565918
train_sentence_count: 32.0
train_sentence_predicted: 9.0
train_sentence_correct: 7.0
train_sentence_total: 26.0
train_sentence_precision: 0.7777777777777778
train_sentence_recall: 0.7777777777777778
train_sentence_f1_score: 0.7777777777777778
train_sentence_f05_score: 0.7777777777777778
train_sentence_correct_binary: 11.0
train_sentence_accuracy_binary: 0.34375
train_tok_0_map: 0.41046325052984306
train_tok_0_p: 0.16444444444444445
train_tok_0_r: 0.44047619047619047
t

train_cost_avg: 0.22219580536087355
train_cost_sum: 85.32318925857544
train_sentence_count: 384.0
train_sentence_predicted: 334.0
train_sentence_correct: 219.0
train_sentence_total: 243.0
train_sentence_precision: 0.655688622754491
train_sentence_recall: 0.655688622754491
train_sentence_f1_score: 0.655688622754491
train_sentence_f05_score: 0.655688622754491
train_sentence_correct_binary: 245.0
train_sentence_accuracy_binary: 0.6380208333333334
train_tok_0_map: 0.34401502092299463
train_tok_0_p: 0.13725490196078433
train_tok_0_r: 0.20933014354066987
train_tok_0_f: 0.16579819990525818
train_tok_0_f05: 0.1474056603773585
train_time: 19.554715156555176
############### Epoch 1 Batch 13 of 898 ###############
train_cost_avg: 0.21886232839180872
train_cost_sum: 91.04672861099243
train_sentence_count: 416.0
train_sentence_predicted: 362.0
train_sentence_correct: 238.0
train_sentence_total: 262.0
train_sentence_precision: 0.6574585635359116
train_sentence_recall: 0.6574585635359116
train_senten

train_cost_avg: 0.20665803800026575
train_cost_sum: 158.7133731842041
train_sentence_count: 768.0
train_sentence_predicted: 649.0
train_sentence_correct: 444.0
train_sentence_total: 485.0
train_sentence_precision: 0.6841294298921418
train_sentence_recall: 0.6841294298921418
train_sentence_f1_score: 0.6841294298921418
train_sentence_f05_score: 0.6841294298921418
train_sentence_correct_binary: 522.0
train_sentence_accuracy_binary: 0.6796875
train_tok_0_map: 0.33546483941507066
train_tok_0_p: 0.1393235739525492
train_tok_0_r: 0.16187683284457477
train_tok_0_f: 0.14975583288117197
train_tok_0_f05: 0.14331706303873715
train_time: 38.2524688243866
############### Epoch 1 Batch 25 of 898 ###############
train_cost_avg: 0.2073541921377182
train_cost_sum: 165.88335371017456
train_sentence_count: 800.0
train_sentence_predicted: 676.0
train_sentence_correct: 460.0
train_sentence_total: 501.0
train_sentence_precision: 0.6804733727810651
train_sentence_recall: 0.6804733727810651
train_sentence_f1_s

train_cost_avg: 0.20547254011034966
train_cost_sum: 236.7043662071228
train_sentence_count: 1152.0
train_sentence_predicted: 938.0
train_sentence_correct: 645.0
train_sentence_total: 718.0
train_sentence_precision: 0.6876332622601279
train_sentence_recall: 0.6876332622601279
train_sentence_f1_score: 0.6876332622601279
train_sentence_f05_score: 0.6876332622601279
train_sentence_correct_binary: 786.0
train_sentence_accuracy_binary: 0.6822916666666666
train_tok_0_map: 0.3359478832234087
train_tok_0_p: 0.1467915956842703
train_tok_0_r: 0.2131958762886598
train_tok_0_f: 0.17386917773667396
train_tok_0_f05: 0.15654332949797128
train_time: 58.10879898071289
############### Epoch 1 Batch 37 of 898 ###############
train_cost_avg: 0.20590850992782697
train_cost_sum: 243.79567575454712
train_sentence_count: 1184.0
train_sentence_predicted: 952.0
train_sentence_correct: 658.0
train_sentence_total: 740.0
train_sentence_precision: 0.6911764705882353
train_sentence_recall: 0.6911764705882353
train_se

KeyboardInterrupt: 

main

In [ ]:
if __name__ == "__main__":
    run_experiment('config.conf')